In [1]:
# importing all libraries

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import sklearn

In [2]:
movie_df = pd.read_csv(r"C:\Users\minautee.m\OneDrive - Fractal Analytics Pvt. Ltd\Documents\Training Material\Recommendation Systems\movies.csv")
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df = pd.read_csv(r"C:\Users\minautee.m\OneDrive - Fractal Analytics Pvt. Ltd\Documents\Training Material\Recommendation Systems\ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
combined_df = movie_df.merge(ratings_df, on='movieId')
combined_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [8]:
user_who_watched_most_movies = combined_df.userId.value_counts()
user_who_watched_most_movies

userId
414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
53       20
207      20
431      20
442      20
189      20
Name: count, Length: 610, dtype: int64

User **414** has watched most number of movies equalling to 2698

In [10]:
most_watched_movie = combined_df['title'].value_counts()
most_watched_movie

title
Forrest Gump (1994)                      329
Shawshank Redemption, The (1994)         317
Pulp Fiction (1994)                      307
Silence of the Lambs, The (1991)         279
Matrix, The (1999)                       278
                                        ... 
We're Back! A Dinosaur's Story (1993)      1
American Hardcore (2006)                   1
Shanghai Surprise (1986)                   1
Let's Get Harry (1986)                     1
Andrew Dice Clay: Dice Rules (1991)        1
Name: count, Length: 9719, dtype: int64

The most watched movie is **Forrest Gump** which has been watched over by 309 different users.

In [20]:
# Creating a pivot table of userId, title and rating

data_pivot =(combined_df.reset_index().pivot_table(index = 'title', columns = 'userId',
              values = 'rating').fillna(0))
data_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
movie_data_matrix = csr_matrix(data_pivot.values)
movie_data_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100832 stored elements and shape (9719, 610)>

In [22]:
model_knn = NearestNeighbors(n_neighbors=5, metric = 'cosine')
model_knn.fit(movie_data_matrix)

NearestNeighbors(metric='cosine')

In [32]:
# Finding the index of the movie of my choice

movie_title = 'Pulp Fiction (1994)'
if movie_title in data_pivot.index:
    query_index = data_pivot.index.get_loc(movie_title)
    print(f"The index of '{movie_title}' is {query_index}.")
else:
    print(f"'{movie_title}' not found in the pivot table.")


The index of 'Pulp Fiction (1994)' is 6865.


In [33]:
query_index= 6865  
distances, indices = model_knn.kneighbors(data_pivot.iloc[query_index, :].values.reshape(1,-1), 
                                            n_neighbors = 6)

In [46]:
# Finding movies similar to Pulp Fiction

print('Movie Recommendations similar to "{0}"\n'.format(data_pivot.index[query_index]))
j=1
for i in indices.flatten()[1:]:
    print('"', data_pivot.index[i],'"', 'with distance of %0.4f' % distances.flatten()[j])
    j+=1

Movie Recommendations similar to "Pulp Fiction (1994)"

" Silence of the Lambs, The (1991) " with distance of 0.2906
" Shawshank Redemption, The (1994) " with distance of 0.2976
" Seven (a.k.a. Se7en) (1995) " with distance of 0.3023
" Forrest Gump (1994) " with distance of 0.3145
" Usual Suspects, The (1995) " with distance of 0.3274


Number of users who have watched the same movie Pulp Fiction similar to my choice

In [44]:
movie_of_interest = 123
users_who_watched = combined_df[combined_df['title'] == 'Pulp Fiction (1994)']['userId'].unique()
users_who_watched

array([  1,   4,   5,   6,   8,  10,  14,  15,  16,  17,  18,  21,  23,
        24,  26,  28,  29,  32,  33,  37,  38,  39,  40,  41,  42,  43,
        45,  50,  54,  56,  57,  58,  62,  63,  64,  66,  67,  68,  69,
        72,  74,  76,  78,  81,  84,  91,  94,  96,  99, 100, 102, 103,
       105, 107, 109, 110, 115, 117, 118, 119, 121, 122, 123, 124, 125,
       126, 130, 131, 132, 133, 134, 135, 136, 137, 140, 141, 142, 144,
       145, 152, 153, 156, 159, 160, 162, 166, 167, 168, 170, 171, 173,
       174, 176, 177, 178, 179, 181, 182, 187, 191, 192, 193, 195, 197,
       198, 199, 200, 201, 202, 203, 204, 210, 211, 212, 215, 217, 219,
       220, 221, 222, 223, 226, 227, 229, 231, 232, 233, 235, 239, 240,
       241, 242, 246, 247, 249, 251, 254, 260, 261, 262, 263, 266, 273,
       274, 275, 279, 282, 284, 285, 286, 288, 290, 292, 294, 295, 296,
       298, 301, 302, 304, 305, 307, 309, 313, 314, 317, 318, 322, 323,
       325, 326, 328, 330, 331, 332, 334, 335, 336, 338, 339, 34